In [2]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [3]:
final_data = []
year = 2016
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

for week in weeks:
    url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    week_data = []
    for board in soup.select('.ScoreboardScoreCell'):
        title = board.find_previous(class_='Card__Header__Title').text
        teams = [t.text for t in board.select('.ScoreCell__TeamName')]
        scores = [s.text for s in board.select('.ScoreCell__Score')] or ['-', '-']

        week_data.append((week, title, teams[0], scores[0], teams[1], scores[1]))

    final_data.extend(week_data)

df_scores_2016 = pd.DataFrame(final_data, columns=['Week', 'Date', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
df_scores_2016 = df_scores_2016.rename(columns={'Team 1': 'Away Team', 'Score 1': 'Away Score', 'Team 2': 'Home Team', 'Score 2':'Home Score'})
df_scores_2016['Final Score'] = df_scores_2016['Away Score'] + ' - ' + df_scores_2016['Home Score']
#print(df.to_markdown(index=False))
# Replace missing scores with NaN and dtype convert object to int 
df_scores_2016['Away Score'] = df_scores_2016['Away Score'].replace('-', np.nan)
df_scores_2016['Home Score'] = df_scores_2016['Home Score'].replace('-', np.nan)
df_scores_2016['Away Score'] = pd.to_numeric(df_scores_2016['Away Score'])
df_scores_2016['Home Score'] =  pd.to_numeric(df_scores_2016['Home Score'])

# Calculate total points
df_scores_2016['Total Points'] = df_scores_2016['Away Score'] + df_scores_2016['Home Score']

# Calculate winner
df_scores_2016['Winner'] = df_scores_2016.apply(lambda row: row['Away Team'] if row['Away Score'] > row['Home Score'] else row['Home Team'], axis=1)

# Calculate score differential
df_scores_2016['Away Score Differential'] = df_scores_2016['Home Score'] - df_scores_2016['Away Score']
df_scores_2016['Home Score Differential'] = df_scores_2016['Away Score'] - df_scores_2016['Home Score']
df_scores_2016['Matchup'] = df_scores_2016['Away Team'] + ' @ ' + df_scores_2016['Home Team']
df_scores_2016['Date'] = pd.to_datetime(df_scores_2016['Date'], format='%A, %B %d, %Y').dt.strftime('%Y-%m-%d')
# Use a lambda function to create the 'Home_Away_Winner' column
df_scores_2016['Home_Away_Winner'] = df_scores_2016.apply(lambda row: 'Home' 
                                                          if row['Winner'] == row['Home Team'] 
                                                          else 'Away' if row['Winner'] == row['Away Team'] else 'No Winner', axis=1)
df_scores_2016.tail()

,Week,Date,Away Team,Away Score,Home Team,Home Score,Final Score,Total Points,Winner,Away Score Differential,Home Score Differential,Matchup,Home_Away_Winner
251,17,2017-01-01,Cardinals,44,Rams,6,44 - 6,50,Cardinals,-38,38,Cardinals @ Rams,Away
252,17,2017-01-01,Chiefs,37,Chargers,27,37 - 27,64,Chiefs,-10,10,Chiefs @ Chargers,Away
253,17,2017-01-01,Seahawks,25,49ers,23,25 - 23,48,Seahawks,-2,2,Seahawks @ 49ers,Away
254,17,2017-01-01,Giants,19,Redskins,10,19 - 10,29,Giants,-9,9,Giants @ Redskins,Away
255,17,2017-01-01,Packers,31,Lions,24,31 - 24,55,Packers,-7,7,Packers @ Lions,Away


In [4]:
#import consensus data for spreads 
game_dates = list(df_scores_2016['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

spreads = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        spreads.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_spreads_2016 = pd.concat(spreads, ignore_index=True)

In [5]:
#clean matchup column 
df_spreads_2016['Matchup'] = df_spreads_2016['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_spreads_2016[['Away Team', 'Home Team']] = df_spreads_2016['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_spreads_2016['Away Team'] = df_spreads_2016['Away Team'].str.strip()
df_spreads_2016['Home Team'] = df_spreads_2016['Home Team'].str.strip()
#split to create away and home spread cols 
df_spreads_2016[['Away Spread', 'Home Spread']] = df_spreads_2016['Sides'].str.split(expand=True)
#split to create away and home consensus cols 
df_spreads_2016[['Away Consensus', 'Home Consensus']] = df_spreads_2016['Consensus'].str.split(expand=True)
# convert to dtype = float for consensus cols 
df_spreads_2016['Away Consensus'] =  df_spreads_2016['Away Consensus'].str.rstrip('%').astype(int)/100
df_spreads_2016['Home Consensus'] = df_spreads_2016['Home Consensus'].str.rstrip('%').astype(int)/100
#create consensus favorite vs. underdog cols 
df_spreads_2016['Con. Spread Favorite'] = df_spreads_2016.apply(lambda x: x['Away Team'] 
                                                                if x['Away Consensus'] > x['Home Consensus'] else x['Home Team'], axis=1)
df_spreads_2016['Con. Spread Underdog'] = df_spreads_2016.apply(lambda x: x['Away Team'] 
                                                                if x['Away Consensus'] < x['Home Consensus'] else x['Home Team'], axis=1)
# Define the dictionary mapping for replacements
replacement_dict = {'LV': 'Raiders',
                    'KC': 'Chiefs',
                    'NE': 'Patriots',
                    'GB': 'Packers',
                    'NO': 'Saints',
                    'BAL': 'Ravens',
                    'SF': '49ers',
                    'NYG': 'Giants',
                    'JAC': 'Jaguars',
                    'PIT': 'Steelers',
                    'TB': 'Buccaneers',
                    'CLE': 'Browns',
                    'IND': 'Colts',
                    'PHI': 'Eagles',
                    'ARI': 'Cardinals',
                    'WAS': 'Commanders',
                    'ATL': 'Falcons',
                    'NYJ': 'Jets',
                    'SEA': 'Seahawks',
                    'MIA': 'Dolphins',
                    'HOU': 'Texans',
                    'CAR': 'Panthers',
                    'CHI': 'Bears',
                    'CIN': 'Bengals',
                    'LAR': 'Rams',
                    'BUF': 'Bills',
                    'DET': 'Lions',
                    'DEN': 'Broncos',
                    'TEN': 'Titans',
                    'MIN': 'Vikings',
                    'LAC': 'Chargers',
                    'DAL': 'Cowboys'}
# Replace the values in the DataFrame column using map()
df_spreads_2016['Away Team'] = df_spreads_2016['Away Team'].map(replacement_dict)
df_spreads_2016['Home Team'] = df_spreads_2016['Home Team'].map(replacement_dict)
df_spreads_2016['Con. Spread Favorite'] = df_spreads_2016['Con. Spread Favorite'].map(replacement_dict)
df_spreads_2016['Con. Spread Underdog'] = df_spreads_2016['Con. Spread Underdog'].map(replacement_dict)
df_spreads_2016['Matchup'] = df_spreads_2016['Away Team'] + ' @ ' + df_spreads_2016['Home Team']
df_spreads_2016.tail()

,Matchup,Date,Consensus,Sides,Picks,Indepth,Away Team,Home Team,Away Spread,Home Spread,Away Consensus,Home Consensus,Con. Spread Favorite,Con. Spread Underdog
251,Texans @ Titans,2017-01-01,58% 42%,+3 -3,1162 838,Details,Texans,Titans,+3,-3,0.58,0.42,Texans,Titans
252,Panthers @ Buccaneers,2017-01-01,43% 57%,+3.5 -3.5,842 1135,Details,Panthers,Buccaneers,+3.5,-3.5,0.43,0.57,Buccaneers,Panthers
253,Cowboys @ Eagles,2017-01-01,56% 44%,+6 -6,1071 841,Details,Cowboys,Eagles,+6,-6,0.56,0.44,Cowboys,Eagles
254,Saints @ Falcons,2017-01-01,54% 46%,+7.5 -7.5,1048 894,Details,Saints,Falcons,+7.5,-7.5,0.54,0.46,Saints,Falcons
255,Bears @ Vikings,2017-01-01,48% 52%,+6 -6,975 1039,Details,Bears,Vikings,+6,-6,0.48,0.52,Vikings,Bears


In [6]:
#import consensus data for spreads 
game_dates = list(df_scores_2016['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

totals = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        totals.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_totals_2016 = pd.concat(totals, ignore_index=True)
df_totals_2016.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth
251,NFL Cle Pit,2017-01-01,55 % Under 45 % Over,41.5,760 631,Details
252,NFL Ari Lar,2017-01-01,53 % Over 47 % Under,39.0,740 653,Details
253,NFL Sea Sf,2017-01-01,53 % Under 47 % Over,43.5,739 661,Details
254,NFL Buf Nyj,2017-01-01,51 % Under 49 % Over,43.0,695 661,Details
255,NFL Chi Min,2017-01-01,51 % Over 49 % Under,44.0,712 690,Details


In [7]:
#clean matchup column 
df_totals_2016['Matchup'] = df_totals_2016['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_totals_2016[['Away Team', 'Home Team']] = df_totals_2016['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_totals_2016['Away Team'] = df_totals_2016['Away Team'].str.strip()
df_totals_2016['Home Team'] = df_totals_2016['Home Team'].str.strip()
# create consensus columns 
df_totals_2016['Over Consensus'] = df_totals_2016['Consensus'].str.extract(r'(\d+) % Over').astype(int)/100
df_totals_2016['Under Consensus'] = df_totals_2016['Consensus'].str.extract(r'(\d+) % Under').astype(int)/100
#create consensus favorites vs underdogs 
df_totals_2016['Con. Total Favorite'] = df_totals_2016[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] > x['Under Consensus'] else 'Under', axis=1)
df_totals_2016['Con. Total Underdog'] = df_totals_2016[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] < x['Under Consensus'] else 'Under', axis=1)
# Replace the values in the DataFrame column using map() from our previously defined replacement_dict
df_totals_2016['Away Team'] = df_totals_2016['Away Team'].map(replacement_dict)
df_totals_2016['Home Team'] = df_totals_2016['Home Team'].map(replacement_dict)
df_totals_2016['Matchup'] = df_totals_2016['Away Team'] + ' @ ' + df_totals_2016['Home Team']
df_totals_2016.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth,Away Team,Home Team,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog
251,Browns @ Steelers,2017-01-01,55 % Under 45 % Over,41.5,760 631,Details,Browns,Steelers,0.45,0.55,Under,Over
252,Cardinals @ Rams,2017-01-01,53 % Over 47 % Under,39.0,740 653,Details,Cardinals,Rams,0.53,0.47,Over,Under
253,Seahawks @ 49ers,2017-01-01,53 % Under 47 % Over,43.5,739 661,Details,Seahawks,49ers,0.47,0.53,Under,Over
254,Bills @ Jets,2017-01-01,51 % Under 49 % Over,43.0,695 661,Details,Bills,Jets,0.49,0.51,Under,Over
255,Bears @ Vikings,2017-01-01,51 % Over 49 % Under,44.0,712 690,Details,Bears,Vikings,0.51,0.49,Over,Under


In [8]:
print(df_scores_2016.columns)
print(df_spreads_2016.columns)
print(df_totals_2016.columns)

Index(['Week', 'Date', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Final Score', 'Total Points', 'Winner', 'Away Score Differential',
       'Home Score Differential', 'Matchup', 'Home_Away_Winner'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Sides', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Total', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog'],
      dtype='object')


In [9]:
# pd.merge(df1, df2, on=['Key_Column_1', 'Key_Column_2'], how='inner')
combined_df = pd.merge(df_scores_2016[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 
                                       'Winner', 'Home_Away_Winner', 'Away Team', 'Away Score', 
                                       'Away Score Differential', 'Home Team', 'Home Score', 'Home Score Differential']], 
                       df_spreads_2016[['Matchup', 'Date', 'Away Team','Away Spread', 'Home Team', 
                                        'Home Spread', 'Away Consensus', 'Home Consensus', 
                                        'Con. Spread Favorite','Con. Spread Underdog']],
                       on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                       how='inner')

In [10]:
final_df_nfl_2016 = pd.merge(combined_df[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
                                           'Home_Away_Winner', 'Away Team', 'Away Score',
                                           'Away Score Differential', 'Home Team', 'Home Score',
                                           'Home Score Differential', 'Away Spread', 'Home Spread',
                                           'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
                                           'Con. Spread Underdog']],
                            df_totals_2016[['Matchup', 'Date', 'Away Team', 'Home Team', 'Total',
                                            'Over Consensus', 'Under Consensus',
                                            'Con. Total Favorite', 'Con. Total Underdog']],
                             on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                             how='inner')

final_df_nfl_2016['Win Marginal'] = np.abs(final_df_nfl_2016['Away Score'] - final_df_nfl_2016['Home Score'])

In [11]:
# 1) did away vs home team cover? -- Done 
# Convert 'Away Spread' and 'Home Spread' columns to numeric values
final_df_nfl_2016['Away Spread'] = pd.to_numeric(final_df_nfl_2016['Away Spread'])
final_df_nfl_2016['Home Spread'] = pd.to_numeric(final_df_nfl_2016['Home Spread'])

final_df_nfl_2016['Spread Covering Team'] = final_df_nfl_2016.apply(lambda row: row['Home Team'] 
                                                              if row['Home Score'] + row['Home Spread'] >= row['Away Score'] 
                                                              else row['Away Team'], axis=1)
final_df_nfl_2016[['Matchup', 'Final Score','Home Team', 'Home Spread', 'Home Score', 'Home Spread','Away Team', 'Away Spread', 'Spread Covering Team']].head()

,Matchup,Final Score,Home Team,Home Spread,Home Score,Home Spread,Away Team,Away Spread,Spread Covering Team
0,Panthers @ Broncos,20 - 21,Broncos,3.0,21,3.0,Panthers,-3.0,Broncos
1,Buccaneers @ Falcons,31 - 24,Falcons,-2.5,24,-2.5,Buccaneers,2.5,Buccaneers
2,Vikings @ Titans,25 - 16,Titans,2.5,16,2.5,Vikings,-2.5,Vikings
3,Chargers @ Chiefs,27 - 33,Chiefs,-6.5,33,-6.5,Chargers,6.5,Chargers
4,Raiders @ Saints,35 - 34,Saints,-2.5,34,-2.5,Raiders,2.5,Raiders


In [12]:
# 2) Did over/under hit? -- Done

# Create "Totals Covered" column
final_df_nfl_2016['Totals Covering Line'] = final_df_nfl_2016.apply(lambda row: 'Over' 
                                          if row['Total Points'] > row['Total'] 
                                          else ('Under' if row['Total Points'] < row['Total'] else 'Push'), axis=1)

# Print the updated DataFrame
final_df_nfl_2016[['Matchup', 'Total Points', 'Total', 'Totals Covering Line']].head()

,Matchup,Total Points,Total,Totals Covering Line
0,Panthers @ Broncos,41,40.5,Over
1,Buccaneers @ Falcons,55,46.5,Over
2,Vikings @ Titans,41,40.0,Over
3,Chargers @ Chiefs,60,45.5,Over
4,Raiders @ Saints,69,50.0,Over


In [13]:
# 3) Did consensus fav vs underdog win? [Spread] -- Done

# Create "Consensus Spread Winner" column
final_df_nfl_2016['Consensus Spread Winner'] = final_df_nfl_2016.apply(lambda row: 'Majority' 
                                            if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] > 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] > 0.5))
                                            else 'Minority' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] < 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] < 0.5))
                                            else 'Push' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] == 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] == 0.5))
                                            else '', axis=1)

# Print the updated DataFrame
final_df_nfl_2016[['Away Team', 'Home Team', 'Away Consensus', 'Home Consensus', 'Consensus Spread Winner']].head()

,Away Team,Home Team,Away Consensus,Home Consensus,Consensus Spread Winner
0,Panthers,Broncos,0.66,0.34,Minority
1,Buccaneers,Falcons,0.40,0.60,Minority
2,Vikings,Titans,0.42,0.58,Minority
3,Chargers,Chiefs,0.42,0.58,Minority
4,Raiders,Saints,0.49,0.51,Minority


In [14]:
# 4) Did consensus fav vs underdog win? [Totals] -- Done
# Create "Total Consensus Winner" column
final_df_nfl_2016['Total Consensus Winner'] = final_df_nfl_2016.apply(lambda row: 'Majority' 
                                                  if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] > 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] > 0.5))
                                                  else 'Minority' if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] < 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] < 0.5))
                                                  else 'Push' if (row['Under Consensus'] == 0.5) or (row['Over Consensus'] == 0.5)
                                                  else '', axis=1)

# Print the updated DataFrame
#final_df_nfl_2016[['Matchup', 'Total Points', 'Total', 'Totals Covering Line', 'Over Consensus', 'Away Consensus', 'Total Consensus Winner']]

In [15]:
final_df_nfl_2016.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner'],
      dtype='object')

In [16]:
final_df_nfl_2016

,Week,Date,Matchup,Final Score,Total Points,Winner,Home_Away_Winner,Away Team,Away Score,Away Score Differential,...,Total,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog,Win Marginal,Spread Covering Team,Totals Covering Line,Consensus Spread Winner,Total Consensus Winner
0,1,2016-09-08,Panthers @ Broncos,20 - 21,41,Broncos,Home,Panthers,20,1,...,40.5,0.37,0.63,Under,Over,1,Broncos,Over,Minority,Minority
1,1,2016-09-11,Buccaneers @ Falcons,31 - 24,55,Buccaneers,Away,Buccaneers,31,-7,...,46.5,0.51,0.49,Over,Under,7,Buccaneers,Over,Minority,Majority
2,1,2016-09-11,Vikings @ Titans,25 - 16,41,Vikings,Away,Vikings,25,-9,...,40.0,0.42,0.58,Under,Over,9,Vikings,Over,Minority,Minority
3,1,2016-09-11,Chargers @ Chiefs,27 - 33,60,Chiefs,Home,Chargers,27,6,...,45.5,0.51,0.49,Over,Under,6,Chargers,Over,Minority,Majority
4,1,2016-09-11,Raiders @ Saints,35 - 34,69,Raiders,Away,Raiders,35,-1,...,50.0,0.65,0.35,Over,Under,1,Raiders,Over,Minority,Majority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,17,2017-01-01,Raiders @ Broncos,6 - 24,30,Broncos,Home,Raiders,6,18,...,40.0,0.44,0.56,Under,Over,18,Broncos,Under,Majority,Majority
236,17,2017-01-01,Cardinals @ Rams,44 - 6,50,Cardinals,Away,Cardinals,44,-38,...,39.0,0.53,0.47,Over,Under,38,Cardinals,Over,Majority,Majority
237,17,2017-01-01,Chiefs @ Chargers,37 - 27,64,Chiefs,Away,Chiefs,37,-10,...,44.5,0.63,0.37,Over,Under,10,Chiefs,Over,Majority,Majority
238,17,2017-01-01,Seahawks @ 49ers,25 - 23,48,Seahawks,Away,Seahawks,25,-2,...,43.5,0.47,0.53,Under,Over,2,49ers,Over,Minority,Minority


In [17]:
final_df_nfl_2016.to_csv('final_df_nfl_2016.csv')